In [30]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [31]:
from src import spotify

In [32]:
token = spotify.get_token()

## Select Tracks

In [39]:
sel_tracks = spotify.search_tracks(token)

sel_tracks.head()

계속하시려면 next, 종료는 exit를 입력해주세요.exit


,id,name,artists,artists id
0,2FWquqPNxte8iqZ3ATQG0p,All I Wanna Do (K) (Feat. Hoody & Loco),Jay Park,4XDi67ZENZcbfKnvMnTYsI
1,4as4XEOR03oGm1STUKl6pa,BBIBBI,IU,3HqSLMAZ3g3d5poNaI7GOU
2,049M1ZP4H2B13yB2ax6N7l,그대라는 시,TAEYEON,3qNVuliS40BLgXGxhdBdqu
3,3mNbGPCLCxia3yvwWp2P51,It Takes Time,Loco,2e4G04F77jxVuDYo44TCSm


## get genres

In [94]:
genres = spotify.get_genres(token, sel_tracks)

genres

,count
k-pop,4
k-rap,2
korean r&b,2


## get features

In [107]:
features = spotify.get_features(token, sel_tracks)

features

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,2FWquqPNxte8iqZ3ATQG0p,0.403,0.724,8,-6.064,1,0.2460,0.377,0.000000,0.0927,0.428,97.133,216786,4
1,4as4XEOR03oGm1STUKl6pa,0.839,0.617,7,-4.692,1,0.3310,0.241,0.000000,0.3000,0.821,143.894,194426,4
2,049M1ZP4H2B13yB2ax6N7l,0.533,0.292,9,-7.100,1,0.0364,0.914,0.000000,0.1180,0.472,135.624,209482,4
3,3mNbGPCLCxia3yvwWp2P51,0.728,0.503,3,-7.147,1,0.0800,0.777,0.000003,0.2490,0.267,120.032,231602,4


## Seed Info

In [110]:
seed_info = pd.merge(left=sel_tracks, right=features, how='inner', on='id')

print("장르 Top : {}".format(genres.index[:5].values))
seed_info.head()

장르 Top : ['k-pop' 'k-rap' 'korean r&b']


,id,name,artists,artists id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,2FWquqPNxte8iqZ3ATQG0p,All I Wanna Do (K) (Feat. Hoody & Loco),Jay Park,4XDi67ZENZcbfKnvMnTYsI,0.403,0.724,8,-6.064,1,0.2460,0.377,0.000000,0.0927,0.428,97.133,216786,4
1,4as4XEOR03oGm1STUKl6pa,BBIBBI,IU,3HqSLMAZ3g3d5poNaI7GOU,0.839,0.617,7,-4.692,1,0.3310,0.241,0.000000,0.3000,0.821,143.894,194426,4
2,049M1ZP4H2B13yB2ax6N7l,그대라는 시,TAEYEON,3qNVuliS40BLgXGxhdBdqu,0.533,0.292,9,-7.100,1,0.0364,0.914,0.000000,0.1180,0.472,135.624,209482,4
3,3mNbGPCLCxia3yvwWp2P51,It Takes Time,Loco,2e4G04F77jxVuDYo44TCSm,0.728,0.503,3,-7.147,1,0.0800,0.777,0.000003,0.2490,0.267,120.032,231602,4


# Recommend